In [1]:
import os
import pandas as pd
import numpy as np
import json
import pprint
import re
import nltk
import csv

In [2]:
data_dir = "data"
data_folders = os.listdir(data_dir)

for data_folder in data_folders:
    print(data_folder,'\n',os.listdir(os.path.join(data_dir,data_folder)))
    print('-'*30)

empatheticdialogues 
 ['test.csv', 'train.csv', 'valid.csv']
------------------------------
goemotions 
 ['goemotions_1.csv', 'goemotions_2.csv', 'goemotions_3.csv']
------------------------------
processed 
 ['all_ed_data.csv', 'ed_for_goemotions.tsv', 'ed_for_goemotions.tsv.eval_results.txt', 'ed_for_goemotions.tsv.label_predictions.tsv', 'ed_for_goemotions.tsv.predictions.tsv', 'ed_for_goemotions.tsv.tf_record', 'all_dd_data.tsv', 'all_meld_data.tsv', 'all_names.csv', 'dd_for_goemotions.tsv', 'meld_for_goemotions.tsv', 'dd_for_goemotions.tsv.eval_results.txt', 'dd_for_goemotions.tsv.label_predictions.tsv', 'dd_for_goemotions.tsv.predictions.tsv', 'dd_for_goemotions.tsv.tf_record', 'meld_for_goemotions.tsv.eval_results.txt', 'meld_for_goemotions.tsv.label_predictions.tsv', 'meld_for_goemotions.tsv.predictions.tsv', 'meld_for_goemotions.tsv.tf_record']
------------------------------
dailydialogues 
 ['dialogues_act.txt', 'dialogues_emotion.txt', 'dialogues_text.txt', 'dialogues_topic.

### Reference for emotion codes

In [50]:
## Use for converting emotion_labels to int
emotion_codes = ("neutral", "anger", "disgust", "fear", "joy", "sadness", "surprise")
emotion_codes = pd.DataFrame(emotion_codes)
emotion_codes

,0
0,neutral
1,anger
2,disgust
3,fear
4,joy
5,sadness
6,surprise


In [43]:
data_folder = '../goemotions/data'
file_name = 'emotions.txt'
emotions = pd.read_csv(os.path.join(data_dir, data_folder, file_name), header=None)
emotions

,0
0,admiration
1,amusement
2,anger
3,annoyance
4,approval
5,caring
6,confusion
7,curiosity
8,desire
9,disappointment


In [47]:
file_name = 'sentiment_dict.json'
with open(os.path.join(data_dir, data_folder, file_name)) as json_data:
    sentiment_map = json.load(json_data)
pprint.pprint(sentiment_map)

{'ambiguous': ['realization', 'surprise', 'curiosity', 'confusion'],
 'negative': ['fear',
              'nervousness',
              'remorse',
              'embarrassment',
              'disappointment',
              'sadness',
              'grief',
              'disgust',
              'anger',
              'annoyance',
              'disapproval'],
 'positive': ['amusement',
              'excitement',
              'joy',
              'love',
              'desire',
              'optimism',
              'caring',
              'pride',
              'admiration',
              'gratitude',
              'relief',
              'approval']}


In [48]:
file_name = 'ekman_mapping.json'
with open(os.path.join(data_dir, data_folder, file_name)) as json_data:
    emotions_group = json.load(json_data)

In [49]:
pprint.pprint(emotions_group)

{'anger': ['anger', 'annoyance', 'disapproval'],
 'disgust': ['disgust'],
 'fear': ['fear', 'nervousness'],
 'joy': ['joy',
         'amusement',
         'approval',
         'excitement',
         'gratitude',
         'love',
         'optimism',
         'relief',
         'pride',
         'admiration',
         'desire',
         'caring'],
 'sadness': ['sadness', 'disappointment', 'embarrassment', 'grief', 'remorse'],
 'surprise': ['surprise', 'realization', 'confusion', 'curiosity']}


### DD Results

In [39]:
data_folder='processed'
file_name='dd_for_goemotions.tsv.label_predictions.tsv'
dd_labels = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t')
print(ed_labels.shape)
dd_labels.head()

(102979, 7)


,text,emotion_1,prob_1,emotion_2,prob_2,emotion_3,prob_3
0,The kitchen stinks.,neutral,0.8440,NaN,NaN,NaN,NaN
1,I 'll throw out the garbage.,neutral,0.2475,approval,0.1503,disappointment,0.0721
2,"So [NAME], how about getting some coffee for t...",curiosity,0.5154,neutral,0.2312,confusion,0.1512
3,Coffee? I don't honestly like that kind of stuff.,curiosity,0.2814,confusion,0.2452,neutral,0.1831
4,"Come on, you can at least try a little, beside...",neutral,0.7220,approval,0.0723,NaN,NaN


In [34]:
file_name='dd_for_goemotions.tsv'
dd_data = pd.read_csv(os.path.join(data_dir, data_folder, file_name), sep='\t', header=None, quoting=csv.QUOTE_NONE)
dd_data.columns = ["utterance", "label", "id"]
print(dd_data.shape)
dd_data.head()

(102979, 3)


,utterance,label,id
0,The kitchen stinks.,2,1_1
1,I 'll throw out the garbage.,0,1_2
2,"So [NAME], how about getting some coffee for t...",4,2_1
3,Coffee? I don't honestly like that kind of stuff.,2,2_2
4,"Come on, you can at least try a little, beside...",0,2_3


In [40]:
dd_data.label = dd_labels.emotion_1
dd_data.head()

,utterance,label,id
0,The kitchen stinks.,neutral,1_1
1,I 'll throw out the garbage.,neutral,1_2
2,"So [NAME], how about getting some coffee for t...",curiosity,2_1
3,Coffee? I don't honestly like that kind of stuff.,curiosity,2_2
4,"Come on, you can at least try a little, beside...",neutral,2_3


In [41]:
ed_data.head(20)

,utterance,label,id
0,"Today, as i was leaving for work in the mornin...",neutral,hit:3_conv:6_1_6
1,Are you fine now?,neutral,hit:3_conv:6_2_7
2,"Yeah, i'm doing alright now, but with minor in...",curiosity,hit:3_conv:6_3_6
3,Cool :) Is your car damaged a lot?,curiosity,hit:3_conv:6_4_7
4,"The car was badly damaged, i veered outside th...",neutral,hit:3_conv:6_5_6
5,"A few weeks ago, I was walking through my hall...",curiosity,hit:4_conv:9_1_8
6,"That's funny, hope he didn't give you a heart ...",neutral,hit:4_conv:9_2_4
7,I may have let out a scream that will have him...,neutral,hit:4_conv:9_3_8
8,I would probably scream also.,neutral,hit:4_conv:9_4_4
9,"""I'm overly excited because will be flying out...",admiration,hit:6_conv:12_1_6


In [11]:
ed_idx

0                  hit:3_conv:6_1_6
1                  hit:3_conv:6_2_7
2                  hit:3_conv:6_3_6
3                  hit:3_conv:6_4_7
4                  hit:3_conv:6_5_6
                    ...            
12073     hit:12361_conv:24722_4_46
12074    hit:12392_conv:24785_1_791
12075    hit:12392_conv:24785_2_829
12076    hit:12392_conv:24785_3_791
12077    hit:12392_conv:24785_4_829
Name: id, Length: 12078, dtype: object